<a href="https://colab.research.google.com/github/sanjay-thiyagarajan/score-predictor/blob/main/CricPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://internalapp.nptel.ac.in/contest/reference_docs/ipl_csv2.zip'
!mkdir '/content/dataset'
!unzip '/content/ipl_csv2.zip' -d '/content/dataset'

In [145]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_parent = []
DATASET_DIR = '/content/dataset/'
for file in os.listdir(DATASET_DIR):
  filename = os.path.join(DATASET_DIR, file)
  df = pd.read_csv(filename, error_bad_lines=False, header=0)
  train_parent.append(df)
train_parent = pd.concat(train_parent, axis=0, ignore_index=True)

In [147]:
people_encoder = LabelEncoder()
people = list(np.array(train_parent[['striker']]).flatten()) + list(np.array(train_parent[['bowler']]).flatten())
people = list(set(people))
people_encoder.fit(people)

LabelEncoder()

In [148]:
train_parent = train_parent.drop(columns=['All Indian Premier League match data in CSV format'])
d = {'runs_off_bat': 'sum', 'striker':', '.join, 'bowler':', '.join}
train_parent = train_parent.groupby(['match_id','innings'], as_index=False).aggregate(d)

In [149]:
strikers = train_parent['striker'].values
bt_unique = []
for s in strikers:
  st = s.split(', ')
  st = list(set(st))
  bt_unique.append(', '.join(st))
bowlers = train_parent['bowler'].values
bw_unique = []
for s in bowlers:
  st = s.split(', ')
  st = list(set(st))
  bw_unique.append(', '.join(st))
train_parent['striker'] = pd.DataFrame(bt_unique)
train_parent['bowler'] = pd.DataFrame(bw_unique)
train_parent.drop(columns=['match_id', 'innings'])

,runs_off_bat,striker,bowler
0,410.0,"BB McCullum, RT Ponting, Mohammad Hafeez, SC G...","Z Khan, JH Kallis, AA Noffke, CL White, SB Jos..."
1,126.0,"Z Khan, JH Kallis, V Kohli, R Dravid, AA Noffk...","I Sharma, AB Agarkar, LR Shukla, AB Dinda, SC ..."
2,468.0,"MEK Hussey, SK Raina, ML Hayden, JDP Oram, MS ...","B Lee, IK Pathan, PP Chawla, K Goel, WA Mota, ..."
3,392.0,"Yuvraj Singh, IK Pathan, SM Katich, K Goel, KC...","Joginder Sharma, JDP Oram, M Muralitharan, MS ..."
4,244.0,"SK Trivedi, DS Lehmann, RA Jadeja, M Rawat, T ...","DL Vettori, MF Maharoof, GD McGrath, R Bhatia,..."
...,...,...,...
1659,296.0,"KA Jamieson, RM Patidar, Shahbaz Ahmed, HV Pat...","TA Boult, KH Pandya, M Jansen, RD Chahar, JJ B..."
1660,356.0,"MM Ali, SK Raina, RA Jadeja, MS Dhoni, AT Rayu...","R Ashwin, CR Woakes, A Mishra, MP Stoinis, Ave..."
1661,372.0,"MP Stoinis, RR Pant, PP Shaw, S Dhawan","SN Thakur, MM Ali, DJ Bravo, RA Jadeja, DL Cha..."
1662,360.0,"EJG Morgan, KD Karthik, N Rana, Shubman Gill, ...","T Natarajan, Rashid Khan, Sandeep Sharma, Moha..."
